### Geoscripting Exercise 14
#### Team Chantalle_Tom
#### Date: 25-01-2018

##### Import Modules

In [1]:
import os, sys, datetime, time, folium, json, string, csv, psycopg2, pprint, urllib
import numpy as np
from twython import Twython
from datetime import datetime
from datetime import date
from time import *

##### Create an "output" and a "data" folder

In [2]:
try:
    os.stat("output")
except:
    os.mkdir("output")
    
try:
    os.stat("data")
except:
    os.mkdir("data")

##### Set up a twython connection to the twitter app

In [3]:
# codes to access twitter API. 
APP_KEY = "IguYRmMdLHICCCRmQnxf2a5FJ"
APP_SECRET = "P47HzAzm5mxmxMVLyx8018ago2Yg90DuvKxivwsvqbJnf1XC2a"
OAUTH_TOKEN = "956457001936179201-HLLsWkifl5qbnlLV6hV58P1EQ9xSJs9"
OAUTH_TOKEN_SECRET = "9Ry0ZYgYah7g1vSPZUpjhAF8BaKyMTeftIc0boQ9hsKGf"

# initiating Twython object
twitter = Twython(APP_KEY, APP_SECRET, OAUTH_TOKEN, OAUTH_TOKEN_SECRET)

#####  Make a twitter search query with hashtag WURcampustour and a maximum count of 100

In [4]:
search_results = twitter.search(q = ["#WURcampustour"], count = 100)

##### Create an array from chosen information from the search result

In [5]:
#parsing out
twitter_data = [] # create empty array
for tweet in search_results["statuses"]:
    username =  tweet['user']['screen_name']
    tweettext = tweet['text']
    
    tweet_data = [username, tweettext]
    
    if tweet['place'] != None:
        full_place_name = tweet['place']['full_name']
        place_type =  tweet['place']['place_type']
        tweet_data.extend([full_place_name, place_type])
        
    if tweet['coordinates'] != None:
        coor = tweet['coordinates']['coordinates']
        tweet_data.extend(coor)
        
    twitter_data += [tweet_data]

del(twitter_data[0],twitter_data[5],twitter_data[14])

##### Store the data into a csv file
The file is stored in the data folder

In [6]:
## store it as a csv file 

output_file = 'data/result_' + datetime.now().strftime('%Y%m%d-%H%M%S') + '.csv'

f = open(output_file, 'w')

with f:
    writer = csv.writer(f)
    for row in twitter_data:
        writer.writerow(row)

#####  Extract the neccesary information from the twitter data (username, message, and coordinates)

In [7]:
# Create an np.array from the twitter data
twitter_data_array = np.array(twitter_data)

# Extract tweet text
tweet_text = twitter_data_array[:,1]

# define function to extract coordinates in proper format for plotting map with folium
def tweet_coor(array):
    tweet_coor = []
    for elem in array:
        elem = sorted([elem[0].astype(float), elem[1].astype(float)], reverse = True)
        tweet_coor += [elem]
    return(tweet_coor)

# extract coordinates array and make the proper plotting format out of it with defined function
tweet_coor_array = twitter_data_array[:,4:6]
tweet_coor_list = tweet_coor(tweet_coor_array)

##### Create a map with the locations of the tweets
The map is stored as an html in the output folder

In [8]:
WUR_map = folium.Map(location=[51.985, 5.665],
                   tiles='openstreetmap', zoom_start = 16)

for i in range(len(tweet_coor_list)):
    folium.Marker(tweet_coor_list[i], popup = tweet_text[i], icon = folium.Icon(color="green", icon="info-sign")).add_to(WUR_map)

WUR_map.save('output/WUR_map.html')